En este Notebook se ejecutan los comandos necesarios para entrenar y evaluar los 4 modelos predictivos (Logistic regression, Standard LSTM, Channel-wise LSTM y Channel-wise Gru) diseñados para la tarea 'Predicción de la mortalidad hospilaria'. 

Nota: Para obtener los archivos en el mortality.zip primero ejecuta los siguientes comandos (para más detalles véase 'pasos_de_comandos_mortality.txt'):

1. python3 -m mimic3benchmark.scripts.extract_subjects {PATH TO MIMIC-III CSVs} data/root/
2. python3 -m mimic3benchmark.scripts.validate_events data/root/
3. python3 -m mimic3benchmark.scripts.extract_episodes_from_subjects data/root/
4. python3 -m mimic3benchmark.scripts.split_train_and_test data/root/
5. python3 -m mimic3benchmark.scripts.create_in_hospital_mortality data/root/ data/{task name}/ 
6. python3 -m mimic3models.create_normalizer_state --impute_strategy previous --n_samples -1 --output_dir . --start_time zero --store_masks --task los --timestep 1.0 --data {PATH TO data/{task name}}
7. python -m mimic3models.split_train_val {dataset-directory}



Observación: el notebook se ejecutó en Google Colab.

### Setups:

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!ls

gdrive	mortality.zip  sample_data


In [ ]:
%cd /content

/content


In [ ]:
!ls

gdrive	mortality.zip  sample_data


Cuidado en subir zip en content: debe mantener en misma pagina, y coprobar que ha subido todo el zip(mirando tamaño de MB)

In [ ]:
!unzip mortality.zip

Se han truncado las últimas 5000 líneas del flujo de salida.
  inflating: in-hospital-mortality/train/4405_episode1_timeseries.csv  
  inflating: in-hospital-mortality/train/6759_episode1_timeseries.csv  
  inflating: in-hospital-mortality/train/20979_episode2_timeseries.csv  
  inflating: in-hospital-mortality/train/17585_episode1_timeseries.csv  
  inflating: in-hospital-mortality/train/11363_episode1_timeseries.csv  
  inflating: in-hospital-mortality/train/31322_episode1_timeseries.csv  
  inflating: in-hospital-mortality/train/11021_episode2_timeseries.csv  
  inflating: in-hospital-mortality/train/13922_episode1_timeseries.csv  
  inflating: in-hospital-mortality/train/58723_episode1_timeseries.csv  
  inflating: in-hospital-mortality/train/18114_episode1_timeseries.csv  
  inflating: in-hospital-mortality/train/4651_episode1_timeseries.csv  
  inflating: in-hospital-mortality/train/10463_episode1_timeseries.csv  
  inflating: in-hospital-mortality/train/8411_episode2_timeseries.

In [ ]:
%cd /content/gdrive/MyDrive/MIMIC_benchmark

/content/gdrive/MyDrive/MIMIC_benchmark


In [ ]:
!ls

50epochsOutputLSTMMortality.txt
data
ihm_results_channelWise_epoch34.json
ihm_results_evaluation_channelWise_epoch14.json
ihm_results_evaluation_logistic.json
ihm_results_lstm_epoch33.json
LICENSE
los3days_results_channelWise_lstm_epoch28.json
los3days_results_channelWise_lstm.json
los3days_results_logistic.json
los3days_results_lstm_epoch35.json
los3days_results_lstm.json
los7days_results_channelWise_lstm.json
los7days_results_logistic.json
los7days_results_lstm.json
los_results_channelWise.json
los_results_logistic.json
los_results_lstm.json
mimic3benchmark
mimic3models
README.md
requirements.txt
scripts_pipeline_forLOS3days.txt
scripts_pipeline_forLOS7days.txt
scripts_pipeline_forLOSCustomized.txt
scripts_pipeline.txt
statistics.md


In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import tensorflow 
print(tensorflow.__version__)

1.15.2


In [ ]:
#Using CPU 
#!pip3 install tensorflow==1.15.2 #no recomenda usar este comando para instalar tensorflow 1.15

In [ ]:
import tensorflow 
print(tensorflow.__version__)

1.15.2


In [ ]:
import tensorflow as tf
tf.test.gpu_device_name() #Testing si esta usando gpu

''

In [ ]:
!pip install keras==2.1.2

  Using cached https://files.pythonhosted.org/packages/68/89/58ee5f56a9c26957d97217db41780ebedca3154392cb903c3f8a08a52208/Keras-2.1.2-py2.py3-none-any.whl
ERROR: textgenrnn 1.4.1 has requirement keras>=2.1.5, but you'll have keras 2.1.2 which is incompatible.
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3


In [ ]:
import keras 
print(keras.__version__)

2.1.2


Using TensorFlow backend.


### Logistic Regression

In [ ]:
!python -um mimic3models.in_hospital_mortality.logistic.main --data /content/mortality --l2 --C 0.001 --output_dir mimic3models/in_hospital_mortality/logistic

Namespace(C=0.001, data='/content/mortality', features='all', l2=True, output_dir='mimic3models/in_hospital_mortality/logistic', period='all')
Reading data and extracting features ...
  train data shape = (14681, 714)
  validation data shape = (3222, 714)
  test data shape = (3236, 714)
Imputing missing values ...
Normalizing the data to have zero mean and unit variance ...
confusion matrix:
[[12420   274]
 [ 1419   568]]
accuracy = 0.8846808671951294
precision class 0 = 0.8974636793136597
precision class 1 = 0.6745843291282654
recall class 0 = 0.9784150123596191
recall class 1 = 0.28585806488990784
AUC of ROC = 0.8632200369044448
AUC of PRC = 0.5507756910072708
min(+P, Se) = 0.5321931589537223
confusion matrix:
[[2707   79]
 [ 315  121]]
accuracy = 0.8777157068252563
precision class 0 = 0.8957644104957581
precision class 1 = 0.6050000190734863
recall class 0 = 0.9716439247131348
recall class 1 = 0.2775229215621948
AUC of ROC = 0.8438901585252606
AUC of PRC = 0.49242911482765134
min(+P

In [ ]:
#Obs. Due to Deprecated warning:
#we have changed the original benchmark version: mimic3models/in_hospital_mortality/logistic/main.py --> sklearn.preprocessing import Imputer, StandardScaler to 
#from sklearn.impute import SimpleImputer 
#and
#imputer = SimpleImputer(missing_values=np.nan, strategy='mean') #insted of 
#from sklearn.preprocessing import Imputer
#imputer = Imputer(missing_values='NaN', strategy='mean', axis=0)

Evaluation:

In [ ]:
!python -m mimic3benchmark.evaluation.evaluate_ihm /content/gdrive/MyDrive/MIMIC_benchmark/mimic3models/in_hospital_mortality/logistic/predictions/all.all.l2.C0.001.csv

Saving the results in ihm_results.json ...
{'n_iters': 10000, 'AUC of ROC': {'value': 0.8485334289995776, 'mean': 0.8484126125931645, 'median': 0.8485996576899298, 'std': 0.010169788254806888, '2.5% percentile': 0.8275049071749977, '97.5% percentile': 0.8678827568735672}, 'AUC of PRC': {'value': 0.48233028943306633, 'mean': 0.4831659385103935, 'median': 0.48303271628113054, 'std': 0.028189741875045818, '2.5% percentile': 0.4272304459838141, '97.5% percentile': 0.5381383389121239}, 'min(+P, Se)': {'value': 0.46684350132625996, 'mean': 0.47141420776219867, 'median': 0.4715447154471545, 'std': 0.022904306207408128, '2.5% percentile': 0.4257696147766037, '97.5% percentile': 0.5158165334260225}}


### Channel wise LSTM

In [ ]:
!python -um mimic3models.in_hospital_mortality.main --data /content/inHospitalMortality/ --network mimic3models/keras_models/channel_wise_lstms.py --dim 8 --depth 1 --batch_size 8 --dropout 0.3 --timestep 1.0 --epochs 55 --mode train --size_coef 4.0 --output_dir mimic3models/in_hospital_mortality

/content/gdrive/My Drive/MIMIC_benchmark/mimic3models/in_hospital_mortality/main.py:7: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
Using TensorFlow backend.
Namespace(batch_norm=False, batch_size=8, beta_1=0.9, data='/content/inHospitalMortality/', depth=1, dim=8, dropout=0.3, epochs=55, imputation='previous', l1=0, l2=0, load_state='', lr=0.001, mode='train', network='mimic3models/keras_models/channel_wise_lstms.py', normalizer_state=None, optimizer='adam', output_dir='mimic3models/in_hospital_mortality', prefix='', rec_dropout=0.0, save_every=1, size_coef=4.0, small_part=False, target_repl_coef=0.0, timestep=1.0, verbose=2)
==> using model mimic3models/keras_models/channel_wise_lstms.py
==> not used params in network class: dict_keys(['batch_size', 'beta_1', 'data', 'epochs', 'imputation', 'l1', 'l2', 'load_state', 'lr', 'mode', 'network', 'normalizer_state', 'optimizer', 'output_dir', 'pref

In [ ]:
!python -um mimic3models.in_hospital_mortality.main --data /content/mortality/ --network mimic3models/keras_models/channel_wise_lstms.py --dim 8 --depth 1 --batch_size 8 --dropout 0.3 --timestep 1.0 --epochs 5 --load_state mimic3models/in_hospital_mortality/keras_states/k_channel_wise_lstms.n8.szc4.0.d0.3.dep1.bs8.ts1.0.epoch9.test0.2846316648826401.state --mode train --size_coef 4.0 --output_dir mimic3models/in_hospital_mortality

/content/gdrive/My Drive/MIMIC_benchmark/mimic3models/in_hospital_mortality/main.py:7: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
Using TensorFlow backend.
Namespace(batch_norm=False, batch_size=8, beta_1=0.9, data='/content/mortality/', depth=1, dim=8, dropout=0.3, epochs=5, imputation='previous', l1=0, l2=0, load_state='mimic3models/in_hospital_mortality/keras_states/k_channel_wise_lstms.n8.szc4.0.d0.3.dep1.bs8.ts1.0.epoch9.test0.2846316648826401.state', lr=0.001, mode='train', network='mimic3models/keras_models/channel_wise_lstms.py', normalizer_state=None, optimizer='adam', output_dir='mimic3models/in_hospital_mortality', prefix='', rec_dropout=0.0, save_every=1, size_coef=4.0, small_part=False, target_repl_coef=0.0, timestep=1.0, verbose=2)
==> using model mimic3models/keras_models/channel_wise_lstms.py
==> not used params in network class: dict_keys(['batch_size', 'beta_1', 'data', 'epo

In [ ]:
!python -um mimic3models.in_hospital_mortality.main --data /content/mortality/ --network mimic3models/keras_models/channel_wise_lstms.py --dim 8 --depth 1 --batch_size 8 --dropout 0.3 --timestep 1.0 --epochs 5 --load_state mimic3models/in_hospital_mortality/keras_states/k_channel_wise_lstms.n8.szc4.0.d0.3.dep1.bs8.ts1.0.epoch14.test0.2786399499460602.state --mode test --size_coef 4.0 --output_dir mimic3models/in_hospital_mortality

/content/gdrive/My Drive/MIMIC_benchmark/mimic3models/in_hospital_mortality/main.py:7: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
Using TensorFlow backend.
Namespace(batch_norm=False, batch_size=8, beta_1=0.9, data='/content/mortality/', depth=1, dim=8, dropout=0.3, epochs=5, imputation='previous', l1=0, l2=0, load_state='mimic3models/in_hospital_mortality/keras_states/k_channel_wise_lstms.n8.szc4.0.d0.3.dep1.bs8.ts1.0.epoch14.test0.2786399499460602.state', lr=0.001, mode='test', network='mimic3models/keras_models/channel_wise_lstms.py', normalizer_state=None, optimizer='adam', output_dir='mimic3models/in_hospital_mortality', prefix='', rec_dropout=0.0, save_every=1, size_coef=4.0, small_part=False, target_repl_coef=0.0, timestep=1.0, verbose=2)
==> using model mimic3models/keras_models/channel_wise_lstms.py
==> not used params in network class: dict_keys(['batch_size', 'beta_1', 'data', 'epo

In [ ]:
!python -m mimic3benchmark.evaluation.evaluate_ihm /content/gdrive/MyDrive/MIMIC_benchmark/mimic3models/in_hospital_mortality/test_predictions/k_channel_wise_lstms.n8.szc4.0.d0.3.dep1.bs8.ts1.0.epoch14.test0.2786399499460602.state.csv

Saving the results in ihm_results.json ...
{'n_iters': 10000, 'AUC of ROC': {'value': 0.8634214882827537, 'mean': 0.8633761045058538, 'median': 0.8636335354866951, 'std': 0.009258945430379579, '2.5% percentile': 0.8447414339327184, '97.5% percentile': 0.8808106839751161}, 'AUC of PRC': {'value': 0.5087855372805268, 'mean': 0.5092538661157673, 'median': 0.509423752054619, 'std': 0.026739082316781206, '2.5% percentile': 0.4572603906175029, '97.5% percentile': 0.5602262257051118}, 'min(+P, Se)': {'value': 0.4946524064171123, 'mean': 0.4949099619054101, 'median': 0.4948453608247423, 'std': 0.02242126732393252, '2.5% percentile': 0.45052018873762373, '97.5% percentile': 0.5371287128712872}}


Load epoch 14 of Channel Wise LSTM and continue training:





In [ ]:
!python -um mimic3models.in_hospital_mortality.main --data /content/mortality/ --network mimic3models/keras_models/channel_wise_lstms.py --dim 8 --depth 1 --batch_size 8 --dropout 0.3 --timestep 1.0 --epochs 5 --load_state mimic3models/in_hospital_mortality/keras_states/k_channel_wise_lstms.n8.szc4.0.d0.3.dep1.bs8.ts1.0.epoch14.test0.2786399499460602.state --mode train --size_coef 4.0 --output_dir mimic3models/in_hospital_mortality

/content/gdrive/My Drive/MIMIC_benchmark/mimic3models/in_hospital_mortality/main.py:7: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
Using TensorFlow backend.
Namespace(batch_norm=False, batch_size=8, beta_1=0.9, data='/content/mortality/', depth=1, dim=8, dropout=0.3, epochs=5, imputation='previous', l1=0, l2=0, load_state='mimic3models/in_hospital_mortality/keras_states/k_channel_wise_lstms.n8.szc4.0.d0.3.dep1.bs8.ts1.0.epoch14.test0.2786399499460602.state', lr=0.001, mode='train', network='mimic3models/keras_models/channel_wise_lstms.py', normalizer_state=None, optimizer='adam', output_dir='mimic3models/in_hospital_mortality', prefix='', rec_dropout=0.0, save_every=1, size_coef=4.0, small_part=False, target_repl_coef=0.0, timestep=1.0, verbose=2)
==> using model mimic3models/keras_models/channel_wise_lstms.py
==> not used params in network class: dict_keys(['batch_size', 'beta_1', 'data', 'ep


Load epoch 19 of Channel Wise LSTM and continue training:

In [ ]:
!python -um mimic3models.in_hospital_mortality.main --data /content/mortality/ --network mimic3models/keras_models/channel_wise_lstms.py --dim 8 --depth 1 --batch_size 8 --dropout 0.3 --timestep 1.0 --epochs 5 --load_state mimic3models/in_hospital_mortality/keras_states/k_channel_wise_lstms.n8.szc4.0.d0.3.dep1.bs8.ts1.0.epoch19.test0.28150085787607376.state --mode train --size_coef 4.0 --output_dir mimic3models/in_hospital_mortality

/content/gdrive/My Drive/MIMIC_benchmark/mimic3models/in_hospital_mortality/main.py:7: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
Using TensorFlow backend.
Namespace(batch_norm=False, batch_size=8, beta_1=0.9, data='/content/mortality/', depth=1, dim=8, dropout=0.3, epochs=5, imputation='previous', l1=0, l2=0, load_state='mimic3models/in_hospital_mortality/keras_states/k_channel_wise_lstms.n8.szc4.0.d0.3.dep1.bs8.ts1.0.epoch19.test0.28150085787607376.state', lr=0.001, mode='train', network='mimic3models/keras_models/channel_wise_lstms.py', normalizer_state=None, optimizer='adam', output_dir='mimic3models/in_hospital_mortality', prefix='', rec_dropout=0.0, save_every=1, size_coef=4.0, small_part=False, target_repl_coef=0.0, timestep=1.0, verbose=2)
==> using model mimic3models/keras_models/channel_wise_lstms.py
==> not used params in network class: dict_keys(['batch_size', 'beta_1', 'data', 'e

Load epoch 24 of Channel Wise LSTM and continue training:

In [ ]:
!python -um mimic3models.in_hospital_mortality.main --data /content/mortality/ --network mimic3models/keras_models/channel_wise_lstms.py --dim 8 --depth 1 --batch_size 8 --dropout 0.3 --timestep 1.0 --epochs 7 --load_state mimic3models/in_hospital_mortality/keras_states/k_channel_wise_lstms.n8.szc4.0.d0.3.dep1.bs8.ts1.0.epoch24.test0.27952357447254667.state --mode train --size_coef 4.0 --output_dir mimic3models/in_hospital_mortality

/content/gdrive/My Drive/MIMIC_benchmark/mimic3models/in_hospital_mortality/main.py:7: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
Using TensorFlow backend.
Namespace(batch_norm=False, batch_size=8, beta_1=0.9, data='/content/mortality/', depth=1, dim=8, dropout=0.3, epochs=7, imputation='previous', l1=0, l2=0, load_state='mimic3models/in_hospital_mortality/keras_states/k_channel_wise_lstms.n8.szc4.0.d0.3.dep1.bs8.ts1.0.epoch24.test0.27952357447254667.state', lr=0.001, mode='train', network='mimic3models/keras_models/channel_wise_lstms.py', normalizer_state=None, optimizer='adam', output_dir='mimic3models/in_hospital_mortality', prefix='', rec_dropout=0.0, save_every=1, size_coef=4.0, small_part=False, target_repl_coef=0.0, timestep=1.0, verbose=2)
==> using model mimic3models/keras_models/channel_wise_lstms.py
==> not used params in network class: dict_keys(['batch_size', 'beta_1', 'data', 'e

Load epoch 30 of Channel Wise LSTM and continue training:


In [ ]:
!python -um mimic3models.in_hospital_mortality.main --data /content/mortality/ --network mimic3models/keras_models/channel_wise_lstms.py --dim 8 --depth 1 --batch_size 8 --dropout 0.3 --timestep 1.0 --epochs 5 --load_state mimic3models/in_hospital_mortality/keras_states/k_channel_wise_lstms.n8.szc4.0.d0.3.dep1.bs8.ts1.0.epoch30.test0.2802050666181772.state --mode train --size_coef 4.0 --output_dir mimic3models/in_hospital_mortality

/content/gdrive/My Drive/MIMIC_benchmark/mimic3models/in_hospital_mortality/main.py:7: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
Using TensorFlow backend.
Namespace(batch_norm=False, batch_size=8, beta_1=0.9, data='/content/mortality/', depth=1, dim=8, dropout=0.3, epochs=5, imputation='previous', l1=0, l2=0, load_state='mimic3models/in_hospital_mortality/keras_states/k_channel_wise_lstms.n8.szc4.0.d0.3.dep1.bs8.ts1.0.epoch30.test0.2802050666181772.state', lr=0.001, mode='train', network='mimic3models/keras_models/channel_wise_lstms.py', normalizer_state=None, optimizer='adam', output_dir='mimic3models/in_hospital_mortality', prefix='', rec_dropout=0.0, save_every=1, size_coef=4.0, small_part=False, target_repl_coef=0.0, timestep=1.0, verbose=2)
==> using model mimic3models/keras_models/channel_wise_lstms.py
==> not used params in network class: dict_keys(['batch_size', 'beta_1', 'data', 'ep

In [ ]:
#Escoger epoch con mejor aucpr en validation set
import pandas as pd
aucpr_channelwise_lstm = pd.read_csv('mimic3models/in_hospital_mortality/keras_logs/k_channel_wise_lstms.n8.szc4.0.d0.3.dep1.bs8.ts1.0.csv', delimiter = ";")
aucpr_channelwise_lstm.head()

,epoch,loss,train_acc,train_auprc,train_auroc,train_minpse,train_prec0,train_prec1,train_rec0,train_rec1,val_acc,val_auprc,val_auroc,val_loss,val_minpse,val_prec0,val_prec1,val_rec0,val_rec1
0,9,0.291532,0.890130,0.584909,0.873141,0.538500,0.900701,0.718970,0.981093,0.309009,0.886406,0.557326,0.859674,0.281022,0.520642,0.901726,0.666667,0.974874,0.321101
1,10,0.291436,0.888700,0.582321,0.871754,0.542526,0.893147,0.786992,0.989680,0.243583,0.884854,0.553593,0.855681,0.287367,0.520642,0.892683,0.721088,0.985284,0.243119
2,11,0.291018,0.889790,0.581246,0.873731,0.536487,0.902120,0.702970,0.978730,0.321590,0.883923,0.558478,0.861057,0.281377,0.519274,0.901198,0.643518,0.972362,0.318807
3,12,0.288937,0.893604,0.594506,0.877154,0.544036,0.906100,0.717949,0.978336,0.352290,0.887027,0.565085,0.860795,0.283618,0.530752,0.905288,0.653846,0.970926,0.350917
4,13,0.287820,0.892310,0.596970,0.877499,0.549296,0.898744,0.772118,0.986608,0.289884,0.885785,0.561174,0.862976,0.278640,0.520642,0.897436,0.688889,0.979900,0.284404


In [ ]:
aucpr_channelwise_lstm['epoch'][aucpr_channelwise_lstm['val_auprc'].idxmax()]+1  #obs. epoch en dataframe empieza por 0, por lo tanto es epoch 34


34

In [ ]:
aucpr_channelwise_lstm['val_auprc'].max()

0.571195055778349

Prediction with epoch 34 on test set:

In [ ]:
!python -um mimic3models.in_hospital_mortality.main --data /content/mortality/ --network mimic3models/keras_models/channel_wise_lstms.py --dim 8 --depth 1 --batch_size 8 --dropout 0.3 --timestep 1.0 --epochs 5 --load_state mimic3models/in_hospital_mortality/keras_states/k_channel_wise_lstms.n8.szc4.0.d0.3.dep1.bs8.ts1.0.epoch34.test0.27684868672373575.state --mode test --size_coef 4.0 --output_dir mimic3models/in_hospital_mortality

/content/gdrive/My Drive/MIMIC_benchmark/mimic3models/in_hospital_mortality/main.py:7: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
Using TensorFlow backend.
Namespace(batch_norm=False, batch_size=8, beta_1=0.9, data='/content/mortality/', depth=1, dim=8, dropout=0.3, epochs=5, imputation='previous', l1=0, l2=0, load_state='mimic3models/in_hospital_mortality/keras_states/k_channel_wise_lstms.n8.szc4.0.d0.3.dep1.bs8.ts1.0.epoch34.test0.27684868672373575.state', lr=0.001, mode='test', network='mimic3models/keras_models/channel_wise_lstms.py', normalizer_state=None, optimizer='adam', output_dir='mimic3models/in_hospital_mortality', prefix='', rec_dropout=0.0, save_every=1, size_coef=4.0, small_part=False, target_repl_coef=0.0, timestep=1.0, verbose=2)
==> using model mimic3models/keras_models/channel_wise_lstms.py
==> not used params in network class: dict_keys(['batch_size', 'beta_1', 'data', 'ep

Boostrapping evaluation with epoch 34 predictions results:

In [ ]:
!python -m mimic3benchmark.evaluation.evaluate_ihm /content/gdrive/MyDrive/MIMIC_benchmark/mimic3models/in_hospital_mortality/test_predictions/k_channel_wise_lstms.n8.szc4.0.d0.3.dep1.bs8.ts1.0.epoch34.test0.27684868672373575.state.csv --save_file ihm_results_channelWise_epoch34.json

Saving the results in ihm_results_channelWise_epoch34.json ...
{'n_iters': 10000, 'AUC of ROC': {'value': 0.8598928612802087, 'mean': 0.8598758497148894, 'median': 0.8600956088157886, 'std': 0.009499060945766736, '2.5% percentile': 0.8411705471007438, '97.5% percentile': 0.8781507664925567}, 'AUC of PRC': {'value': 0.5192039911553818, 'mean': 0.5190758935235442, 'median': 0.5196955233889695, 'std': 0.026055253456478175, '2.5% percentile': 0.46706581964982086, '97.5% percentile': 0.5684961731992673}, 'min(+P, Se)': {'value': 0.48128342245989303, 'mean': 0.48310733324586363, 'median': 0.4832904884318766, 'std': 0.02243314914408363, '2.5% percentile': 0.43820224719101125, '97.5% percentile': 0.525886088012311}}


### Standard LSTM:

In [ ]:
!python -um mimic3models.in_hospital_mortality.main --network mimic3models/keras_models/lstm.py --data /content/mortality/ --dim 16 --epochs 35 --timestep 1.0 --depth 2 --dropout 0.3 --mode train --batch_size 8 --output_dir mimic3models/in_hospital_mortality --verbose 1

/content/gdrive/My Drive/MIMIC_benchmark/mimic3models/in_hospital_mortality/main.py:7: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
Using TensorFlow backend.
Namespace(batch_norm=False, batch_size=8, beta_1=0.9, data='/content/mortality/', depth=2, dim=16, dropout=0.3, epochs=35, imputation='previous', l1=0, l2=0, load_state='', lr=0.001, mode='train', network='mimic3models/keras_models/lstm.py', normalizer_state=None, optimizer='adam', output_dir='mimic3models/in_hospital_mortality', prefix='', rec_dropout=0.0, save_every=1, size_coef=4.0, small_part=False, target_repl_coef=0.0, timestep=1.0, verbose=1)
==> using model mimic3models/keras_models/lstm.py
==> not used params in network class: dict_keys(['batch_size', 'beta_1', 'data', 'epochs', 'imputation', 'l1', 'l2', 'load_state', 'lr', 'mode', 'network', 'normalizer_state', 'optimizer', 'output_dir', 'prefix', 'save_every', 'size_coef', 'smal

In [ ]:
#Escoger epoch con mejor aucpr en validation set
import pandas as pd
aucpr_lstm = pd.read_csv('mimic3models/in_hospital_mortality/keras_logs/k_lstm.n16.d0.3.dep2.bs8.ts1.0.csv', delimiter = ";")
aucpr_lstm.head()

,epoch,loss,train_acc,train_auprc,train_auroc,train_minpse,train_prec0,train_prec1,train_rec0,train_rec1,val_acc,val_auprc,val_auroc,val_loss,val_minpse,val_prec0,val_prec1,val_rec0,val_rec1
0,0,0.354242,0.873374,0.471574,0.825720,0.486160,0.879935,0.651659,0.988420,0.138400,0.871198,0.456206,0.825358,0.310406,0.471526,0.879117,0.610526,0.986719,0.133028
1,1,0.321763,0.878823,0.497510,0.833633,0.498746,0.897343,0.609937,0.970931,0.290388,0.876164,0.490401,0.829798,0.304796,0.486301,0.898232,0.582222,0.966260,0.300459
2,2,0.316750,0.880049,0.514240,0.841355,0.509311,0.897419,0.622030,0.972428,0.289884,0.880199,0.502407,0.838155,0.299592,0.493363,0.898142,0.620192,0.971644,0.295872
3,3,0.310535,0.882433,0.525585,0.843710,0.508551,0.891379,0.695067,0.983929,0.234021,0.881440,0.515974,0.841727,0.296106,0.505643,0.892810,0.666667,0.980617,0.247706
4,4,0.308207,0.883659,0.535117,0.850411,0.514343,0.902890,0.633238,0.969750,0.333669,0.879578,0.517827,0.843789,0.294265,0.514806,0.903704,0.595238,0.963388,0.344037


In [ ]:
aucpr_lstm['epoch'][aucpr_lstm['val_auprc'].idxmax()]+1  #obs. index empieza por 0, por lo tanto es epoch 33


33

In [ ]:
aucpr_lstm['val_auprc'].max()

0.5575143349575479

Prediction on test set with epoch 33

In [ ]:
!python -um mimic3models.in_hospital_mortality.main --network mimic3models/keras_models/lstm.py --data /content/mortality/ --dim 16 --timestep 1.0 --depth 2 --dropout 0.3 --batch_size 8 --load_state mimic3models/in_hospital_mortality/keras_states/k_lstm.n16.d0.3.dep2.bs8.ts1.0.epoch33.test0.28255336389898295.state --output_dir mimic3models/in_hospital_mortality --mode test


/content/gdrive/My Drive/MIMIC_benchmark/mimic3models/in_hospital_mortality/main.py:7: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
Using TensorFlow backend.
Namespace(batch_norm=False, batch_size=8, beta_1=0.9, data='/content/mortality/', depth=2, dim=16, dropout=0.3, epochs=100, imputation='previous', l1=0, l2=0, load_state='mimic3models/in_hospital_mortality/keras_states/k_lstm.n16.d0.3.dep2.bs8.ts1.0.epoch33.test0.28255336389898295.state', lr=0.001, mode='test', network='mimic3models/keras_models/lstm.py', normalizer_state=None, optimizer='adam', output_dir='mimic3models/in_hospital_mortality', prefix='', rec_dropout=0.0, save_every=1, size_coef=4.0, small_part=False, target_repl_coef=0.0, timestep=1.0, verbose=2)
==> using model mimic3models/keras_models/lstm.py
==> not used params in network class: dict_keys(['batch_size', 'beta_1', 'data', 'epochs', 'imputation', 'l1', 'l2', 'load_state'

Boostrapping evaluation:

In [ ]:
!python -m mimic3benchmark.evaluation.evaluate_ihm /content/gdrive/MyDrive/MIMIC_benchmark/mimic3models/in_hospital_mortality/test_predictions/k_lstm.n16.d0.3.dep2.bs8.ts1.0.epoch33.test0.28255336389898295.state.csv --save_file ihm_results_lstm_epoch33.json

Saving the results in ihm_results_lstm_epoch33.json ...
{'n_iters': 10000, 'AUC of ROC': {'value': 0.8520625231224566, 'mean': 0.851994403085846, 'median': 0.8521508533020403, 'std': 0.009978554219682743, '2.5% percentile': 0.8318327637734352, '97.5% percentile': 0.8710023694902397}, 'AUC of PRC': {'value': 0.487215081244672, 'mean': 0.4876163602755445, 'median': 0.48767376517403066, 'std': 0.027005753847682595, '2.5% percentile': 0.43375157059806524, '97.5% percentile': 0.5400936342106492}, 'min(+P, Se)': {'value': 0.4666666666666667, 'mean': 0.46398545110773665, 'median': 0.4642857142857143, 'std': 0.022603084418975974, '2.5% percentile': 0.41818181818181815, '97.5% percentile': 0.506821804647072}}


### Channel wise GRU

In [ ]:
!python -um mimic3models.in_hospital_mortality.main --data /content/inHospitalMortality/ --network mimic3models/keras_models/channel_wise_grus.py --dim 8 --depth 1 --batch_size 8 --dropout 0.3 --timestep 1.0 --epochs 5 --mode train --size_coef 4.0 --output_dir mimic3models/in_hospital_mortality

/content/gdrive/My Drive/MIMIC_benchmark/mimic3models/in_hospital_mortality/main.py:7: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
Using TensorFlow backend.
Namespace(batch_norm=False, batch_size=8, beta_1=0.9, data='/content/inHospitalMortality/', depth=1, dim=8, dropout=0.3, epochs=5, imputation='previous', l1=0, l2=0, load_state='', lr=0.001, mode='train', network='mimic3models/keras_models/channel_wise_grus.py', normalizer_state=None, optimizer='adam', output_dir='mimic3models/in_hospital_mortality', prefix='', rec_dropout=0.0, save_every=1, size_coef=4.0, small_part=False, target_repl_coef=0.0, timestep=1.0, verbose=2)
==> using model mimic3models/keras_models/channel_wise_grus.py
==> not used params in network class: dict_keys(['batch_size', 'beta_1', 'data', 'epochs', 'imputation', 'l1', 'l2', 'load_state', 'lr', 'mode', 'network', 'normalizer_state', 'optimizer', 'output_dir', 'prefix'

Load epoch 5 of Channel Wise GRU and continue training:


In [ ]:
!python -um mimic3models.in_hospital_mortality.main --data /content/inHospitalMortality/ --network mimic3models/keras_models/channel_wise_grus.py --dim 8 --depth 1 --batch_size 8 --dropout 0.3 --timestep 1.0 --epochs 5 --load_state mimic3models/in_hospital_mortality/keras_states/k_channel_wise_grus.n8.szc4.0.d0.3.dep1.bs8.ts1.0.epoch5.test0.2880334512166989.state --mode train --size_coef 4.0 --output_dir mimic3models/in_hospital_mortality

/content/gdrive/My Drive/MIMIC_benchmark/mimic3models/in_hospital_mortality/main.py:7: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
Using TensorFlow backend.
Namespace(batch_norm=False, batch_size=8, beta_1=0.9, data='/content/inHospitalMortality/', depth=1, dim=8, dropout=0.3, epochs=5, imputation='previous', l1=0, l2=0, load_state='mimic3models/in_hospital_mortality/keras_states/k_channel_wise_grus.n8.szc4.0.d0.3.dep1.bs8.ts1.0.epoch5.test0.2880334512166989.state', lr=0.001, mode='train', network='mimic3models/keras_models/channel_wise_grus.py', normalizer_state=None, optimizer='adam', output_dir='mimic3models/in_hospital_mortality', prefix='', rec_dropout=0.0, save_every=1, size_coef=4.0, small_part=False, target_repl_coef=0.0, timestep=1.0, verbose=2)
==> using model mimic3models/keras_models/channel_wise_grus.py
==> not used params in network class: dict_keys(['batch_size', 'beta_1', 'data

In [ ]:
#5 epochs 1h49min, cada epoch aprox. 22 minutos

In [ ]:
!python -um mimic3models.in_hospital_mortality.main --data /content/inHospitalMortality/ --network mimic3models/keras_models/channel_wise_grus.py --dim 8 --depth 1 --batch_size 8 --dropout 0.3 --timestep 1.0 --epochs 5 --load_state mimic3models/in_hospital_mortality/keras_states/k_channel_wise_grus.n8.szc4.0.d0.3.dep1.bs8.ts1.0.epoch10.test0.2866723704955949.state --mode train --size_coef 4.0 --output_dir mimic3models/in_hospital_mortality

/content/gdrive/My Drive/MIMIC_benchmark/mimic3models/in_hospital_mortality/main.py:7: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
Using TensorFlow backend.
Namespace(batch_norm=False, batch_size=8, beta_1=0.9, data='/content/inHospitalMortality/', depth=1, dim=8, dropout=0.3, epochs=5, imputation='previous', l1=0, l2=0, load_state='mimic3models/in_hospital_mortality/keras_states/k_channel_wise_grus.n8.szc4.0.d0.3.dep1.bs8.ts1.0.epoch10.test0.2866723704955949.state', lr=0.001, mode='train', network='mimic3models/keras_models/channel_wise_grus.py', normalizer_state=None, optimizer='adam', output_dir='mimic3models/in_hospital_mortality', prefix='', rec_dropout=0.0, save_every=1, size_coef=4.0, small_part=False, target_repl_coef=0.0, timestep=1.0, verbose=2)
==> using model mimic3models/keras_models/channel_wise_grus.py
==> not used params in network class: dict_keys(['batch_size', 'beta_1', 'dat

In [ ]:
!python -um mimic3models.in_hospital_mortality.main --data /content/inHospitalMortality/ --network mimic3models/keras_models/channel_wise_grus.py --dim 8 --depth 1 --batch_size 8 --dropout 0.3 --timestep 1.0 --epochs 5 --load_state mimic3models/in_hospital_mortality/keras_states/k_channel_wise_grus.n8.szc4.0.d0.3.dep1.bs8.ts1.0.epoch15.test0.2814401373091431.state --mode train --size_coef 4.0 --output_dir mimic3models/in_hospital_mortality

/content/gdrive/My Drive/MIMIC_benchmark/mimic3models/in_hospital_mortality/main.py:7: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
Using TensorFlow backend.
Namespace(batch_norm=False, batch_size=8, beta_1=0.9, data='/content/inHospitalMortality/', depth=1, dim=8, dropout=0.3, epochs=5, imputation='previous', l1=0, l2=0, load_state='mimic3models/in_hospital_mortality/keras_states/k_channel_wise_grus.n8.szc4.0.d0.3.dep1.bs8.ts1.0.epoch15.test0.2814401373091431.state', lr=0.001, mode='train', network='mimic3models/keras_models/channel_wise_grus.py', normalizer_state=None, optimizer='adam', output_dir='mimic3models/in_hospital_mortality', prefix='', rec_dropout=0.0, save_every=1, size_coef=4.0, small_part=False, target_repl_coef=0.0, timestep=1.0, verbose=2)
==> using model mimic3models/keras_models/channel_wise_grus.py
==> not used params in network class: dict_keys(['batch_size', 'beta_1', 'dat

In [ ]:
!python -um mimic3models.in_hospital_mortality.main --data /content/inHospitalMortality/ --network mimic3models/keras_models/channel_wise_grus.py --dim 8 --depth 1 --batch_size 8 --dropout 0.3 --timestep 1.0 --epochs 8 --load_state mimic3models/in_hospital_mortality/keras_states/k_channel_wise_grus.n8.szc4.0.d0.3.dep1.bs8.ts1.0.epoch20.test0.2798298893740353.state --mode train --size_coef 4.0 --output_dir mimic3models/in_hospital_mortality

/content/gdrive/My Drive/MIMIC_benchmark/mimic3models/in_hospital_mortality/main.py:7: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
Using TensorFlow backend.
Namespace(batch_norm=False, batch_size=8, beta_1=0.9, data='/content/inHospitalMortality/', depth=1, dim=8, dropout=0.3, epochs=8, imputation='previous', l1=0, l2=0, load_state='mimic3models/in_hospital_mortality/keras_states/k_channel_wise_grus.n8.szc4.0.d0.3.dep1.bs8.ts1.0.epoch20.test0.2798298893740353.state', lr=0.001, mode='train', network='mimic3models/keras_models/channel_wise_grus.py', normalizer_state=None, optimizer='adam', output_dir='mimic3models/in_hospital_mortality', prefix='', rec_dropout=0.0, save_every=1, size_coef=4.0, small_part=False, target_repl_coef=0.0, timestep=1.0, verbose=2)
==> using model mimic3models/keras_models/channel_wise_grus.py
==> not used params in network class: dict_keys(['batch_size', 'beta_1', 'dat

In [ ]:
!python -um mimic3models.in_hospital_mortality.main --data /content/inHospitalMortality/ --network mimic3models/keras_models/channel_wise_grus.py --dim 8 --depth 1 --batch_size 8 --dropout 0.3 --timestep 1.0 --epochs 7 --load_state mimic3models/in_hospital_mortality/keras_states/k_channel_wise_grus.n8.szc4.0.d0.3.dep1.bs8.ts1.0.epoch28.test0.27845172514684596.state --mode train --size_coef 4.0 --output_dir mimic3models/in_hospital_mortality

/content/gdrive/My Drive/MIMIC_benchmark/mimic3models/in_hospital_mortality/main.py:7: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
Using TensorFlow backend.
Namespace(batch_norm=False, batch_size=8, beta_1=0.9, data='/content/inHospitalMortality/', depth=1, dim=8, dropout=0.3, epochs=7, imputation='previous', l1=0, l2=0, load_state='mimic3models/in_hospital_mortality/keras_states/k_channel_wise_grus.n8.szc4.0.d0.3.dep1.bs8.ts1.0.epoch28.test0.27845172514684596.state', lr=0.001, mode='train', network='mimic3models/keras_models/channel_wise_grus.py', normalizer_state=None, optimizer='adam', output_dir='mimic3models/in_hospital_mortality', prefix='', rec_dropout=0.0, save_every=1, size_coef=4.0, small_part=False, target_repl_coef=0.0, timestep=1.0, verbose=2)
==> using model mimic3models/keras_models/channel_wise_grus.py
==> not used params in network class: dict_keys(['batch_size', 'beta_1', 'da

In [17]:
#Escoger epoch con mejor aucpr en validation set
import pandas as pd
aucpr_channelwise_gru = pd.read_csv('mimic3models/in_hospital_mortality/keras_logs/k_channel_wise_grus.n8.szc4.0.d0.3.dep1.bs8.ts1.0.csv', delimiter = ";")
aucpr_channelwise_gru.head()

,epoch,loss,train_acc,train_auprc,train_auroc,train_minpse,train_prec0,train_prec1,train_rec0,train_rec1,val_acc,val_auprc,val_auroc,val_loss,val_minpse,val_prec0,val_prec1,val_rec0,val_rec1
0,0,0.346344,0.878074,0.493122,0.827492,0.482394,0.884214,0.700611,0.988420,0.173125,0.873060,0.478242,0.826350,0.306816,0.497706,0.881052,0.631068,0.986360,0.149083
1,1,0.328046,0.880730,0.513224,0.840483,0.497232,0.888463,0.697987,0.985820,0.209361,0.876785,0.499349,0.838852,0.299855,0.503417,0.887447,0.640288,0.982053,0.204128
2,2,0.319170,0.882842,0.541680,0.852767,0.517363,0.887938,0.748603,0.989365,0.202315,0.880509,0.532387,0.846405,0.292807,0.514806,0.886884,0.714286,0.987796,0.194954
3,3,0.310205,0.887133,0.551500,0.857300,0.510563,0.901141,0.678571,0.976603,0.315551,0.882682,0.532220,0.847229,0.292500,0.496568,0.902406,0.626087,0.969131,0.330275
4,4,0.305934,0.888836,0.562991,0.862218,0.524912,0.896544,0.742156,0.985111,0.273780,0.887337,0.544422,0.853372,0.288033,0.510251,0.897343,0.710983,0.982053,0.282110


In [18]:
aucpr_channelwise_gru['epoch'][aucpr_channelwise_gru['val_auprc'].idxmax()]+1  #obs. epoch en dataframe empieza por 0


20

In [19]:
aucpr_channelwise_gru['val_auprc'].max()

0.580738091413438

In [21]:
!python -um mimic3models.in_hospital_mortality.main --data /content/inHospitalMortality/ --network mimic3models/keras_models/channel_wise_grus.py --dim 8 --depth 1 --batch_size 8 --dropout 0.3 --timestep 1.0 --load_state mimic3models/in_hospital_mortality/keras_states/k_channel_wise_grus.n8.szc4.0.d0.3.dep1.bs8.ts1.0.epoch20.test0.2798298893740353.state --mode test --size_coef 4.0 --output_dir mimic3models/in_hospital_mortality

/content/gdrive/My Drive/MIMIC_benchmark/mimic3models/in_hospital_mortality/main.py:7: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
Using TensorFlow backend.
Namespace(batch_norm=False, batch_size=8, beta_1=0.9, data='/content/inHospitalMortality/', depth=1, dim=8, dropout=0.3, epochs=100, imputation='previous', l1=0, l2=0, load_state='mimic3models/in_hospital_mortality/keras_states/k_channel_wise_grus.n8.szc4.0.d0.3.dep1.bs8.ts1.0.epoch20.test0.2798298893740353.state', lr=0.001, mode='test', network='mimic3models/keras_models/channel_wise_grus.py', normalizer_state=None, optimizer='adam', output_dir='mimic3models/in_hospital_mortality', prefix='', rec_dropout=0.0, save_every=1, size_coef=4.0, small_part=False, target_repl_coef=0.0, timestep=1.0, verbose=2)
==> using model mimic3models/keras_models/channel_wise_grus.py
==> not used params in network class: dict_keys(['batch_size', 'beta_1', 'da

Boostrapping evaluation with epoch 34 predictions results:

In [22]:
!python -m mimic3benchmark.evaluation.evaluate_ihm /content/gdrive/MyDrive/MIMIC_benchmark/mimic3models/in_hospital_mortality/test_predictions/k_channel_wise_grus.n8.szc4.0.d0.3.dep1.bs8.ts1.0.epoch20.test0.2798298893740353.state.csv --save_file ihm_results_channelWise_gru_epoch20.json

Saving the results in ihm_results_channelWise_gru_epoch20.json ...
{'n_iters': 10000, 'AUC of ROC': {'value': 0.8574147879086838, 'mean': 0.8574392534672739, 'median': 0.8576042428060806, 'std': 0.0094947635939188, '2.5% percentile': 0.8383498897136623, '97.5% percentile': 0.8756705981133133}, 'AUC of PRC': {'value': 0.5137592813485251, 'mean': 0.5140281415993513, 'median': 0.5141655638556487, 'std': 0.026347873079304827, '2.5% percentile': 0.4609384367268379, '97.5% percentile': 0.5650138370516915}, 'min(+P, Se)': {'value': 0.4789473684210526, 'mean': 0.4796626125876457, 'median': 0.4797687861271676, 'std': 0.02253666367027177, '2.5% percentile': 0.43420960489458943, '97.5% percentile': 0.5231966639780203}}
